In [66]:
!pip install segmentation_models_pytorch

Необходимо подготовить датасет https://www.kaggle.com/olekslu/makeup-lips-segmentation-28k-samples для обучения модели на сегментацию губ

Обучить модель на выбор из segmentation_models_pytorchfiles_names_list = sorted(os.listdir(images_path))

In [67]:
import os
import time

import numpy as np
import pandas as pd

from pathlib import Path
from PIL import Image

from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

import cv2

import torch
from torch import nn, optim
from torch.nn import functional as F
from torch.utils.data import Dataset, DataLoader

import segmentation_models_pytorch as smp

In [68]:
dataset_file = '/kaggle/input/makeup-lips-segmentation-28k-samples/set-lipstick-original/list.csv'
images_path = '/kaggle/input/makeup-lips-segmentation-28k-samples/set-lipstick-original/720p/'
masks_path = '/kaggle/input/makeup-lips-segmentation-28k-samples/set-lipstick-original/mask/'

In [69]:
df = pd.read_csv(dataset_file)
df.head()

In [70]:
files_names_list = sorted(os.listdir(images_path))
masks_names_list = sorted(os.listdir(masks_path))
print(f'Images {len(files_names_list)}')
print(f'Masks {len(masks_names_list)}')

In [71]:
#почистим лист -- уберем строки с файлами без масок
df = df.loc[df['mask'].isin([i for i in masks_names_list])]
df = df.loc[df['filename'].isin([i for i in files_names_list])]
df.reset_index(drop=True, inplace=True)

In [72]:
#посмотрим данные
image = cv2.imread(images_path + files_names_list[11], cv2.COLOR_BGR2RGB)
mask = cv2.imread(masks_path + masks_names_list[11], cv2.COLOR_BGR2RGB)

fig, ax = plt.subplots(1, 2, figsize=(10, 10))
ax[0].imshow(image)
ax[0].set_title(f'Изображение {image.shape}')
ax[1].imshow(mask)
ax[1].set_title(f'Маска {mask.shape}')
plt.show()

In [73]:
data = df[['filename', 'mask']]
data.head()

In [74]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
device

In [75]:
#инициализация модели
segmodel = smp.Linknet(encoder_name = 'resnet34', 
                    encoder_weights = 'imagenet', 
                    in_channels = 3, 
                    classes = 1, # число каналов маски
                    activation = 'sigmoid').to(device)

preprocess_input = smp.encoders.get_preprocessing_fn('resnet34', pretrained='imagenet')

In [76]:
#класс датасета
class CustomDataset(Dataset):
    def __init__(self, data, preprocessing=None):
        # загружаем данные из датасета
        self.data = data
        self.preprocessing = preprocessing
        self.image_arr = self.data.iloc[:,0] # имена файлов картинок из датасета
        self.label_arr = self.data.iloc[:,1] # имена файлов масок из датасета
        self.data_len = len(self.data.index)
        
    def __len__(self):
        return self.data_len
        
    def __getitem__(self, index):
       # загружаем картинки
        img = cv2.imread(images_path+self.image_arr[index]) 
        img = cv2.cvtColor(cv2.resize(img, (256, 256)), cv2.COLOR_BGR2RGB)
        img = np.asarray(img).astype('float')
        
        if self.preprocessing:
            img = self.preprocessing(img)
            img = torch.as_tensor(img)
        
        else:
            img = torch.as_tensor(img)
            img /= 255.0
        
        img = img.permute(2,0,1)
        
        # загружаем маски
        mask = cv2.imread(masks_path+self.label_arr[index])
        mask = cv2.cvtColor(mask, cv2.COLOR_BGR2RGB)        
        cls_mask = np.where(mask > 50, 1, 0)[:,:,1]
        cls_mask = cls_mask.astype('float')
        cls_mask = cv2.resize(cls_mask, (256, 256))

        masks = [cls_mask]
        masks = torch.as_tensor(masks, dtype=torch.uint8)
        
        
        return (img.float(), masks)

In [77]:
#разделение датасета на train и test
X_train, X_test = train_test_split(data, test_size=0.3, random_state=42)

#сбрасываем индексы
X_train.reset_index(drop=True,inplace=True)
X_test.reset_index(drop=True,inplace=True)

#создаем датасеты
train_data = CustomDataset(X_train, preprocessing=preprocess_input)
test_data = CustomDataset(X_test, preprocessing=preprocess_input)

#создаем даталодеры
train_data_loader = DataLoader(train_data, batch_size=8, shuffle=True)
test_data_loader = DataLoader(test_data, batch_size=4, shuffle=False)

In [78]:
#просмотр данных из даталоадера
for img, target in train_data_loader:
    print(img.shape, target.shape)
    print(img[0].min(), img[0].max())
    print(target[0].min(), target[0].max())
    fig, ax = plt.subplots(1, 2, figsize=(15, 6))
    ax[0].imshow(img[0].permute(1, 2, 0))
    ax[1].imshow(target[0].permute(1, 2, 0)[..., 0])
    break

In [79]:
#определение критерия, метрики и оптимизатора
criterion = smp.utils.losses.DiceLoss()
metrics = [smp.utils.metrics.IoU(),]
optimizer = torch.optim.Adam(params=segmodel.parameters(), lr=0.001)

#определение тренировочных и валидационных эпох
train_epoch = smp.utils.train.TrainEpoch(
    segmodel, 
    loss=criterion, 
    metrics=metrics, 
    optimizer=optimizer,
    device=device,
    verbose=True)

valid_epoch = smp.utils.train.ValidEpoch(
    segmodel, 
    loss=criterion, 
    metrics=metrics,
    device=device,
    verbose=True)


In [80]:
# обучение модели
num_epochs = 10
linknet_train_iou = []
linknet_valid_iou = []

for i in range(num_epochs):
    print(f'Epoch: {i + 1}')
    train_logs = train_epoch.run(train_data_loader)
    valid_logs = valid_epoch.run(test_data_loader)
    
    linknet_train_iou.append(train_logs['iou_score'])
    linknet_valid_iou.append(valid_logs['iou_score'])

In [81]:
x = np.arange(1,11,1)
y = linknet_train_iou
y2 = linknet_valid_iou

plt.plot(x,y,label='IOU_train')
plt.plot(x,y2,label='IOU_val')

plt.title(f"Linknet")   
plt.ylabel('IOU_score')   
plt.xlabel('epochs')   
plt.legend()
plt.show()

In [90]:
#предсказание модели
for i, data in enumerate(test_data_loader):
    images, labels = data
    images = images.to(device)
    labels = labels.to(device)
    
    outputs = segmodel(images)
    
    f, axarr = plt.subplots(1,3, figsize=(15, 6))

    for j in range(0, 4):
        axarr[0].imshow(outputs.detach().cpu()[i, ].permute(1, 2, 0))
        axarr[0].set_title('Guessed labels')
        
        axarr[1].imshow(labels[i, ].cpu().permute(1, 2, 0))
        axarr[1].set_title('Ground truth labels')
        
        image = images[i].permute(1, 2, 0)
        axarr[2].imshow(image.cpu())
        axarr[2].set_title('Original Images')
        plt.show()
    if i > 2:
        break